In [1]:
# SETUP PATHS

WORKSPACE_PATH = 'workspace'
APIMODEL_PATH = 'models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = PRETRAINED_MODEL_PATH+'/pipeline.config'

In [2]:
# Loading the saved_model
import tensorflow as tf
import os
import time
import sys
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from PIL import Image
import pathlib
import pandas as pd

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

In [3]:
### change the location accordingly
detection_model = tf.saved_model.load(MODEL_PATH+'/model/inference_graph/saved_model')

#Loading the label_map
PATH_TO_LABELS = ANNOTATION_PATH+'/anotasi/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [4]:
category_index

{1: {'id': 1, 'name': 'car'},
 2: {'id': 2, 'name': 'motorcycle'},
 3: {'id': 3, 'name': 'person'},
 4: {'id': 4, 'name': 'pothole'}}

In [5]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
#   print(output_dict)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
#   print(output_dict['detection_classes'])
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.8,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict


def show_inference(model, image_np):
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)

#   print(category_index)
  # Visualization of the results of a detection.
  final_img =vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,      
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=0.50, 
          line_thickness=8)
  list_classes = output_dict['detection_classes'].tolist()
  list_scores = output_dict['detection_scores'].tolist()
  index_max_score = list_scores.index(max(output_dict['detection_scores']))
  if list_scores[0] > 0.50:
      print(category_index[list_classes[0]]['name'], list_scores[0]*100)
  return(final_img)
#   display(Image.fromarray(image_np))

In [6]:
# import cv2
# from threading import Thread
# import time

# class ThreadedCamera(object):
#     def __init__(self, source = 0):

#         self.capture = cv2.VideoCapture(source)

#         self.thread = Thread(target = self.update, args = ())
#         self.thread.daemon = True
#         self.thread.start()

#         self.status = False
#         self.frame  = None

#     def update(self):
#         while True:
#             if self.capture.isOpened():
#                 (self.status, self.frame) = self.capture.read()

#     def grab_frame(self):
#         if self.status:
#             return self.frame
#         return None
# if __name__ == '__main__':
# #     stream_link = 'tcp://193.168.0.1:6200/'
#     vid_link= 'workspace/images/motorcycle/melintas.mp4'
#     streamer = ThreadedCamera(vid_link)
    
#     prev_frame_time = 0
#     prev_frame_time = 0

#     while True:
#         img = streamer.grab_frame()
#         if img is not None:
#             img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#             final_img = show_inference(detection_model,img)
            
#             new_frame_time = time.time()
#             fps = 1/(new_frame_time-prev_frame_time)
#             prev_frame_time = new_frame_time
#             fps = str(fps)
# #             print('fps: ',fps)
    
#             final_img = cv2.cvtColor(final_img,cv2.COLOR_RGB2BGR)
#             cv2.imshow('object detection', cv2.resize(final_img, (1000, 600)))

#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     streamer.release()
#     cv2.destroyAllWindows()

In [10]:
import cv2

# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("workspace/images/motorcycle/melintas.mp4")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while 1:
    _,img = cap.read()
    
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    final_img = show_inference(detection_model,img)
    
    final_img = cv2.cvtColor(final_img,cv2.COLOR_RGB2BGR)

    cv2.imshow('object detection', cv2.resize(final_img, (1000, 600)))
#     cv2.imshow('img',img)
#     cv2.imshow('Object Detection',final_img)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

person 52.444714307785034
person 55.41685223579407
person 54.80864644050598
person 57.227784395217896
person 51.429712772369385
person 62.01736927032471
person 57.4243426322937
person 52.92897820472717
person 50.60076117515564
person 57.61599540710449
motorcycle 71.15488648414612
motorcycle 88.61511945724487
motorcycle 81.91692233085632
motorcycle 76.45947337150574
motorcycle 58.24745297431946
motorcycle 67.30813980102539
motorcycle 91.69935584068298
motorcycle 94.97051239013672
motorcycle 96.89061045646667
motorcycle 89.71806168556213
motorcycle 91.15243554115295
motorcycle 89.53891396522522
motorcycle 81.00905418395996
motorcycle 82.26399421691895
motorcycle 95.45232057571411
motorcycle 94.68223452568054
motorcycle 89.72479104995728
motorcycle 96.35006189346313
motorcycle 94.05617713928223
motorcycle 89.423668384552
motorcycle 87.45128512382507
motorcycle 84.72368717193604
motorcycle 84.7745418548584
motorcycle 73.48418235778809
motorcycle 73.03702235221863
motorcycle 75.799816846847

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
